In [1]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("BasicDFOperationsApp").master("spark://spark-master:7077").getOrCreate() 

Spark can infer the schema from the dataset.

In [2]:
df = spark.read.format("json").load("/home/jovyan/data/2015-summary.json")
df.schema

StructType([StructField('DEST_COUNTRY_NAME', StringType(), True), StructField('ORIGIN_COUNTRY_NAME', StringType(), True), StructField('count', LongType(), True)])

We can also create a schema explicitly and use it to load and validate the dataset. The recommendation is to use an explicit schema because schema inference can lead to precision issues (e.g., inferring Integer type as long type) 

In [3]:
from pyspark.sql.types import StructField, StructType, StringType, LongType

explicitSchema = StructType([
 StructField("DEST_COUNTRY_NAME", StringType(), True),
 StructField("ORIGIN_COUNTRY_NAME", StringType(), True),
 StructField("count", LongType(), False, metadata={"somemetaname":"somemetavalue"})
])

dfes = spark.read.format("json").schema(explicitSchema)\
          .load("/home/jovyan/data/2015-summary.json")
dfes.show(5)

+-----------------+-------------------+-----+
|DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|count|
+-----------------+-------------------+-----+
|    United States|            Romania|   15|
|    United States|            Croatia|    1|
|    United States|            Ireland|  344|
|            Egypt|      United States|   15|
|    United States|              India|   62|
+-----------------+-------------------+-----+
only showing top 5 rows



We can select a set of records from a data frame using column names. 

In [4]:
newdf = df.select("DEST_COUNTRY_NAME", "ORIGIN_COUNTRY_NAME")
newdf.show(3)

+-----------------+-------------------+
|DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|
+-----------------+-------------------+
|    United States|            Romania|
|    United States|            Croatia|
|    United States|            Ireland|
+-----------------+-------------------+
only showing top 3 rows



We can use "expr" function as the parameters of "select". To further simplify, Spark provides a new function selectExpr.

In [5]:
from pyspark.sql.functions import expr
# Both the following expressions are doing the same.
df.select(expr("DEST_COUNTRY_NAME as target")).show(2)

df.selectExpr("DEST_COUNTRY_NAME as target").show(2)

+-------------+
|       target|
+-------------+
|United States|
|United States|
+-------------+
only showing top 2 rows

+-------------+
|       target|
+-------------+
|United States|
|United States|
+-------------+
only showing top 2 rows



In [6]:
# Add a new column. Use a Boolean expression for generating values for the column 
df.selectExpr(
    "*", # all original columns
    "(DEST_COUNTRY_NAME = ORIGIN_COUNTRY_NAME) as withinCountry")\
.show(20)

+--------------------+-------------------+-----+-------------+
|   DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|count|withinCountry|
+--------------------+-------------------+-----+-------------+
|       United States|            Romania|   15|        false|
|       United States|            Croatia|    1|        false|
|       United States|            Ireland|  344|        false|
|               Egypt|      United States|   15|        false|
|       United States|              India|   62|        false|
|       United States|          Singapore|    1|        false|
|       United States|            Grenada|   62|        false|
|          Costa Rica|      United States|  588|        false|
|             Senegal|      United States|   40|        false|
|             Moldova|      United States|    1|        false|
|       United States|       Sint Maarten|  325|        false|
|       United States|   Marshall Islands|   39|        false|
|              Guyana|      United States|   64|       

We can add columns. lit function creates a Column of a literal value. Using Spark withColumn() function we can add , rename , derive, split, etc a Dataframe Column

In [7]:
from pyspark.sql.functions import lit

df.withColumn("OneConst", lit(1)).show(2)

+-----------------+-------------------+-----+--------+
|DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|count|OneConst|
+-----------------+-------------------+-----+--------+
|    United States|            Romania|   15|       1|
|    United States|            Croatia|    1|       1|
+-----------------+-------------------+-----+--------+
only showing top 2 rows



We can also rename columns. 

In [8]:
df1 = df.withColumnRenamed("DEST_COUNTRY_NAME", "target")
df1.columns

['target', 'ORIGIN_COUNTRY_NAME', 'count']

We can also remove columns. 

In [9]:
df2 = df.drop("ORIGIN_COUNTRY_NAME")
df2.columns

['DEST_COUNTRY_NAME', 'count']

We can also cast data types.

In [10]:
from pyspark.sql.functions import col
df.withColumn("countlong", col("count").cast("long")).drop("count").schema

StructType([StructField('DEST_COUNTRY_NAME', StringType(), True), StructField('ORIGIN_COUNTRY_NAME', StringType(), True), StructField('countlong', LongType(), True)])

We can filter the rows using the where function, similar to SQL where clause.

In [11]:
df.where("count < 2").show(2)

+-----------------+-------------------+-----+
|DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|count|
+-----------------+-------------------+-----+
|    United States|            Croatia|    1|
|    United States|          Singapore|    1|
+-----------------+-------------------+-----+
only showing top 2 rows



We can select a subset of data, similar to SQL SELECT statement.

In [12]:
df.select("ORIGIN_COUNTRY_NAME").count()
df.select("ORIGIN_COUNTRY_NAME").distinct().count()

125

We can take a random sample from a dataset.

In [13]:
seed = 5
withReplacement = False
fraction = 0.5
df.sample(withReplacement, fraction, seed).count()

138

We can randomly split a dataset into multiple subsets. 

In [14]:
seed = 5
weights = [0.25, 0.75]
dataFrames = df.randomSplit(weights, seed)
dataFrames[0].count() > dataFrames[1].count() # False

False

We can order or sort a dataset in ascending or descending order, similar to SQL Order By clause.
Sort vs OrderBy https://towardsdatascience.com/sort-vs-orderby-in-spark-8a912475390

In [15]:
from pyspark.sql.functions import desc, asc, col, expr

df.sort("count").show(50)
df.orderBy("count", "DEST_COUNTRY_NAME").show(5)
df.orderBy(col("count"), col("DEST_COUNTRY_NAME")).show(5)
df.orderBy(expr("count desc")).show(2)
df.orderBy(col("count").desc(), col("DEST_COUNTRY_NAME").asc()).show(2)


+--------------------+-------------------+-----+
|   DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|count|
+--------------------+-------------------+-----+
|       United States|            Croatia|    1|
|       United States|          Singapore|    1|
|             Moldova|      United States|    1|
|               Malta|      United States|    1|
|       United States|          Gibraltar|    1|
|Saint Vincent and...|      United States|    1|
|            Suriname|      United States|    1|
|       United States|             Cyprus|    1|
|        Burkina Faso|      United States|    1|
|            Djibouti|      United States|    1|
|       United States|            Estonia|    1|
|              Zambia|      United States|    1|
|              Cyprus|      United States|    1|
|       United States|          Lithuania|    1|
|       United States|           Bulgaria|    1|
|       United States|            Georgia|    1|
|       United States|            Bahrain|    1|
|       Cote d'Ivoir

We can limit the size of a dataframe (result). 

In [16]:
df.limit(5).show()

+-----------------+-------------------+-----+
|DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|count|
+-----------------+-------------------+-----+
|    United States|            Romania|   15|
|    United States|            Croatia|    1|
|    United States|            Ireland|  344|
|            Egypt|      United States|   15|
|    United States|              India|   62|
+-----------------+-------------------+-----+



We can collect the data to our driver program. 

In [17]:
collectDF = df.limit(10)
collectDF.take(5) # take works with an Integer count
collectDF.show() # this prints it out nicely
collectDF.show(5, False) # prints only 5. truncate=
collectDF.collect() # gets all data

+-----------------+-------------------+-----+
|DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|count|
+-----------------+-------------------+-----+
|    United States|            Romania|   15|
|    United States|            Croatia|    1|
|    United States|            Ireland|  344|
|            Egypt|      United States|   15|
|    United States|              India|   62|
|    United States|          Singapore|    1|
|    United States|            Grenada|   62|
|       Costa Rica|      United States|  588|
|          Senegal|      United States|   40|
|          Moldova|      United States|    1|
+-----------------+-------------------+-----+

+-----------------+-------------------+-----+
|DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|count|
+-----------------+-------------------+-----+
|United States    |Romania            |15   |
|United States    |Croatia            |1    |
|United States    |Ireland            |344  |
|Egypt            |United States      |15   |
|United States    |India         

[Row(DEST_COUNTRY_NAME='United States', ORIGIN_COUNTRY_NAME='Romania', count=15),
 Row(DEST_COUNTRY_NAME='United States', ORIGIN_COUNTRY_NAME='Croatia', count=1),
 Row(DEST_COUNTRY_NAME='United States', ORIGIN_COUNTRY_NAME='Ireland', count=344),
 Row(DEST_COUNTRY_NAME='Egypt', ORIGIN_COUNTRY_NAME='United States', count=15),
 Row(DEST_COUNTRY_NAME='United States', ORIGIN_COUNTRY_NAME='India', count=62),
 Row(DEST_COUNTRY_NAME='United States', ORIGIN_COUNTRY_NAME='Singapore', count=1),
 Row(DEST_COUNTRY_NAME='United States', ORIGIN_COUNTRY_NAME='Grenada', count=62),
 Row(DEST_COUNTRY_NAME='Costa Rica', ORIGIN_COUNTRY_NAME='United States', count=588),
 Row(DEST_COUNTRY_NAME='Senegal', ORIGIN_COUNTRY_NAME='United States', count=40),
 Row(DEST_COUNTRY_NAME='Moldova', ORIGIN_COUNTRY_NAME='United States', count=1)]

In [18]:
# Stop the spark context
spark.stop()